# Web Scraping From SET

setting to authorize this chromdrive can be opened

command : xattr -d com.apple.quarantine \<name-of-executable\>

In [ ]:
!xattr -d com.apple.quarantine chromedriver

## Web scraping

In [10]:
import pandas as pd
from selenium import  webdriver
import os

### Connect to chrome

In [4]:
driver = webdriver.Chrome(os.getcwd() + "/chromedriver")
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="cb093f316d4d97db749b74a061d810b8")>

### Collect all company names from SET

In [ ]:
def get_all_company_names() : 
    list_page = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "NUMBER"]
    df_all_com_names = pd.DataFrame()

    # open set.or.th to collect comppany names into dataframe
    for p in list_page :
        driver.get("https://classic.set.or.th/set/commonslookup.do?language=th&country=TH&prefix=" + p)
        page = driver.page_source #HTML
        df_page = pd.read_html(page)[0] #HTML to Dataframe
        df_all_com_names = df_all_com_names.append(df_page)
    
    # transform
    df_all_com_names.set_index("ชื่อย่อหลักทรัพย์", inplace=True)

    # return dataframe
    return df_all_com_names

    

In [ ]:
df_all_com_names = get_all_company_names()

check company names

In [ ]:
df_all_com_names.index

### Collect stock information of each company

In [ ]:
# this symbols are difference format. it's stock fund.
format_info_error = ["CPNCG", "CPTGF", "ERWPF", "KPNPF", "LPF", "M-II", "M-STOR", "MNIT2", "POPF", "PPF", "SIRIP", "SSPF", "TLHPF", "TTLPF"]

df_com_names = df_all_com_names.drop(index=[format_info_error])

In [ ]:
def get_stock_information() :   
    
    import time
    start = time.time()
    print("Start!!")
    
    stock_information = pd.DataFrame()
    stocks = df_com_names.index
    
    for stock in stocks :
        if stock == "F&D" : 
            stock = "F%26D"
        elif stock == "L&E" :
            stock = "L%26E"
            
        url = f"https://classic.set.or.th/set/factsheet.do?symbol={stock}&ssoPageId=3&language=th&country=TH"
        driver.get(url)
        factsheet = driver.page_source
        
        # as = alias (stock symbol)
        as_df = pd.read_html(factsheet)[2]
        as_df.drop(columns=1, inplace=True)
        as_df.columns = ["ชื่อย่อหลักทรัพย์"]
        
        if pd.read_html(factsheet, index_col=0)[9].iloc[0][1] == "ข้อมูลเบื้องต้น" :
            # a_df = information
            a_df = pd.read_html(factsheet, index_col=0)[9]
            a_df = a_df.transpose()
            a_df = a_df.iloc[[0]]
            a_df = a_df[["ที่อยู่", "เบอร์โทรศัพท์", "เบอร์โทรสาร", "URL", "พาร์"]]
            a_df.reset_index(drop=True, inplace=True)
        
        elif pd.read_html(factsheet, index_col=0)[9].iloc[0][1] == "ข่าวล่าสุด" :
            # a_df = information
            a_df = pd.read_html(factsheet, index_col=0)[10]
            a_df = a_df.transpose()
            a_df = a_df.iloc[[0]]
            a_df = a_df[["ที่อยู่", "เบอร์โทรศัพท์", "เบอร์โทรสาร", "URL", "พาร์"]]
            a_df.reset_index(drop=True, inplace=True)
        

        # b_df = industrial / category
        b_df = pd.read_html(factsheet)[5]
        b_df = b_df[0].str.split(r"/", expand=True)
        b_df.rename(columns={0 : "กลุ่มอุตสาหกรรม", 1 : "หมวดธุรกิจ"}, inplace=True)
        b_df = b_df.iloc[[0]]
        
        
        if pd.read_html(factsheet, index_col=0)[18].iloc[0][1] == "สะสม: อัตราหมุนเวียนปริมาณการซื้อขาย, มูลค่าซื้อขาย/วัน" :
            # c_df = statistical
            c_df = pd.read_html(factsheet, index_col=0)[18]
            c_df = c_df.transpose()
            c_df.dropna(axis = 1, how = 'all', inplace=True)
            c_df = c_df.iloc[[0]]
            c_df.drop(columns=["สะสม: อัตราหมุนเวียนปริมาณการซื้อขาย, มูลค่าซื้อขาย/วัน", "ข้อมูลสถิติ", "อัตราผลตอบแทน"], inplace=True)
            c_df.reset_index(drop=True, inplace=True)
            
            # d_bg_df = biger shareholder
            d_df = pd.read_html(factsheet)[16]
            d_df.drop(columns=0, inplace=True)
            d_df.columns = [d_df[1][0], d_df[2][0], d_df[3][0]]
            d_df.drop(index=0, inplace=True)
            d_df.reset_index(drop=True, inplace=True)
            d_bg_df = d_df.iloc[[0]]
            d_bg_df.columns = ["ผู้ถือหุ้นรายใหญ่", "จำนวนหุ้น", "สัดส่วนหุ้น"]

        
        elif pd.read_html(factsheet, index_col=0)[18].iloc[0][1] == "คณะกรรมการ" :
            # c_df = statistical
            c_df = pd.read_html(factsheet, index_col=0)[19]
            c_df = c_df.transpose()
            c_df.dropna(axis = 1, how = 'all', inplace=True)
            c_df = c_df.iloc[[0]]
            c_df.drop(columns=["สะสม: อัตราหมุนเวียนปริมาณการซื้อขาย, มูลค่าซื้อขาย/วัน", "ข้อมูลสถิติ", "อัตราผลตอบแทน"], inplace=True)
            c_df.reset_index(drop=True, inplace=True)

            # d_bg_df = biger shareholder
            d_df = pd.read_html(factsheet)[17]
            d_df.drop(columns=0, inplace=True)
            d_df.columns = [d_df[1][0], d_df[2][0], d_df[3][0]]
            d_df.drop(index=0, inplace=True)
            d_df.reset_index(drop=True, inplace=True)
            d_bg_df = d_df.iloc[[0]]
            d_bg_df.columns = ["ผู้ถือหุ้นรายใหญ่", "จำนวนหุ้น", "สัดส่วนหุ้น"]
            
    
        elif pd.read_html(factsheet, index_col=0)[18].iloc[0][1] == "การจ่ายปันผล" :
            # c_df = statistical
            c_df = pd.read_html(factsheet, index_col=0)[17]
            c_df = c_df.transpose()
            c_df.dropna(axis = 1, how = 'all', inplace=True)
            c_df = c_df.iloc[[0]]
            c_df.drop(columns=["สะสม: อัตราหมุนเวียนปริมาณการซื้อขาย, มูลค่าซื้อขาย/วัน", "ข้อมูลสถิติ", "อัตราผลตอบแทน"], inplace=True)
            c_df.reset_index(drop=True, inplace=True)

            # d_bg_df = biger shareholder
            d_df = pd.read_html(factsheet)[15]
            d_df.drop(columns=0, inplace=True)
            d_df.columns = [d_df[1][0], d_df[2][0], d_df[3][0]]
            d_df.drop(index=0, inplace=True)
            d_df.reset_index(drop=True, inplace=True)
            d_bg_df = d_df.iloc[[0]]
            d_bg_df.columns = ["ผู้ถือหุ้นรายใหญ่", "จำนวนหุ้น", "สัดส่วนหุ้น"]
            
            
        elif pd.read_html(factsheet, index_col=0)[18].iloc[0][1] == "นางสาว กมลทิพย์ เลิศวิทย์วรเทพ/บริษัท สำนักงาน อีวาย จำกัด  นาย วิชาติ โลเกศกระวี/บริษัท สำนักงาน อีวาย จำกัด  นางสาว สาธิดา รัตนานุรักษ์/บริษัท สำนักงาน อีวาย จำกัด  นางสาว ศิริวรรณ นิตย์ดำรง/บริษัท สำนักงาน อีวาย จำกัด" :
            # c_df = statistical
            c_df = pd.read_html(factsheet, index_col=0)[16]
            c_df = c_df.transpose()
            c_df.dropna(axis = 1, how = 'all', inplace=True)
            c_df = c_df.iloc[[0]]
            c_df.drop(columns=["สะสม: อัตราหมุนเวียนปริมาณการซื้อขาย, มูลค่าซื้อขาย/วัน", "ข้อมูลสถิติ", "อัตราผลตอบแทน"], inplace=True)
            c_df.reset_index(drop=True, inplace=True)

            # d_bg_df = biger shareholder
            d_df = pd.read_html(factsheet)[15]
            d_df.drop(columns=0, inplace=True)
            d_df.columns = [d_df[1][0], d_df[2][0], d_df[3][0]]
            d_df.drop(index=0, inplace=True)
            d_df.reset_index(drop=True, inplace=True)
            d_bg_df = d_df.iloc[[0]]
            d_bg_df.columns = ["ผู้ถือหุ้นรายใหญ่", "จำนวนหุ้น", "สัดส่วนหุ้น"]

        
        # e_df = financail highlight
        driver.get(f"https://classic.set.or.th/set/companyhighlight.do?symbol={stock}&language=th&country=TH")
        finance = driver.page_source
        e_df = pd.read_html(finance,index_col=0)[0]
        e_df = e_df.iloc[1:9]
        e_df = e_df.transpose()
        e_df = e_df.iloc[[-1]]
        e_df.reset_index(drop=True, inplace=True)
        
        
        # join dataframe
        info = as_df.join([b_df, a_df, c_df, d_bg_df, e_df])

        # append to stock_information
        stock_information = stock_information.append(info)
         
    end = time.time()
    print("Done!!!", end - start, "second")
        
    return stock_information


In [ ]:
stock_information = get_stock_information()

In [ ]:
stock_information

In [ ]:
stock_information.to_csv(os.getcwd() + "/Output/company_names.csv", index=False, encoding="utf-8")